![DLI Header](../images/DLI_Header.png)

# Bonus Material: Senders and Receivers

In this section we demonstrate how to simplify the implementation from the previous section by using _senders_ and _receivers_ with a `static_thread_pool`, giving coverage to the design described in [proposal P2300](https://www.open-std.org/jtc1/sc22/wg21/docs/papers/2022/p2300r5.html), using NVIDIA's [reference implementation](https://github.com/NVIDIA/stdexec).

---

## Refactor Overview

The full implementation of our application using senders and receivers can be found at [solutions/exercise3.cpp](solutions/exercise3.cpp).

Here we give a summary of the refactor.

### Previous Implementation

The implementation from the previous section is quite complex. It requires:

```c++
// A shared atomic variable to accumulate the energy:
std::atomic<double> energy = 0.;

// A shared barrier for synchronizing threads:
std::barrier bar(3);

// User must manually create and start threads:
std::thread thread_inner(..[&] {
      energy += computation(...);
      bar.arrive_and_wait();
      // User must manually create a critical section for MPI rank reduction: 
      MPI_Reduce(...);
      // User must manually reset the shared state on each iteration:
      energy = 0;
      bar.arrive_and_wait();
  });

std::thread thread_prev(...);
std::thread thread_next(...);

// User must manually join all threads before doing File I/O
thread_prev.join();
thread_next.join();
thread_inner.join();

// File I/O
```

### Senders and Receivers

Instead, we use senders and receivers to create a graph representing the computation...

```c++
stde::sender iteration_step(stde::scheduler sch, parameters p, long it,
                            std::vector<double>& u_new, std::vector<double>& u_old) {
    // TODO: use Senders & Receivers to create a graph representing the computation of a single iteration   
}
```

... and then dispatch it to an execution context:

```c++
stde::static_thread_pool ctx{3}; // Thread Pool with 3 threads
stde::scheduler auto sch = ctx.get_scheduler();

for (long it = 0; it < p.nit(); ++it) {
    stde::this_thread::sync_wait(iteration_step(sch));
}
```

### Full Implementation

Please review the [full implementation](solutions/exercise3.cpp) before proceeding to compile and run the refactored application.

---

## Visualize Function

As in the previous sections, we will utilize the `visualize` function to observe the output of running the `heat` application. Execute the cell below to define it for use in this notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

#plt.style.use('dark_background') # Uncomment for dark background

def visualize(name = 'output'):
    f = open(name, 'rb')
    grid = np.fromfile(f, dtype=np.uint64, count=2, offset=0)

    nx = grid[0]
    ny = grid[1]

    times = np.fromfile(f, dtype=np.float64, count=1, offset=0)
    time = times[0]

    values = np.fromfile(f, dtype=np.float64, offset=0)
    assert len(values) == nx * ny, f'{len(values)} != {nx * ny}'
    values = values.reshape((nx, ny))

    print(f'Plotting grid {nx}x{ny}, t = {time}')

    plt.title(f'Temperature at t = {time:.3f} [s]')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.pcolormesh(values, cmap=plt.cm.jet, vmin=0, vmax=values.max())
    plt.colorbar()
    plt.savefig('output.png', transparent=True, bbox_inches='tight', dpi=300)

---

## Compile and Run Commands

The following blocks compile and run the application using different compilers and C++ standard versions.

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=g++ mpicxx -std=c++20 -Ofast -march=native -DNDEBUG -o heat solutions/exercise3.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000
visualize()

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=clang++ mpicxx -std=c++20 -Ofast -march=native -DNDEBUG -o heat solutions/exercise3.cpp -ltbb
!mpirun -np 2 ./heat 256 256 16000
visualize()

In [ ]:
!rm output || true
!rm heat || true
!OMPI_CXX=nvc++ mpicxx -std=c++20 -stdpar=gpu -O4 -fast -march=native -Mllvm-fast -DNDEBUG -o heat solutions/exercise3.cpp
!mpirun -np 2 ./heat 256 256 16000
# As with the previous sections, you might decide to bump the parameters to something like 2048 2048 16000
# to better observe the impact of GPU acceleration.
visualize()

---

## Conclusion

We hope you have enjoyed learning more about GPU acceleration using the C++ Standard Library, and in particular, feel prepared to refactor your multi-node applications using MPI to make the most of this powerful C++ feature set.

![DLI Header](../images/DLI_Header.png)